In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests

In [5]:
url = "https://humanist.kdl.kcl.ac.uk/"
result = requests.get(url)

print(result.status_code)
print(result.headers)
page = result.content

# How would we get all the links to volumes from the Humanist Page?

soup = BeautifulSoup(page)

200
{'Server': 'ATS', 'Date': 'Wed, 23 Jun 2021 05:25:40 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '5814', 'X-Frame-Options': 'SAMEORIGIN', 'Age': '0', 'Connection': 'keep-alive'}


https://humanist.kdl.kcl.ac.uk/Archives/Current/ https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/

In [3]:
links = ['https://humanist.kdl.kcl.ac.uk/Archives/Current/','https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/']
texts = []
for link in links:
    result = requests.get(link)
    page = result.content

    soup = BeautifulSoup(page)
    text_links = soup.find_all('a')
    for txt_link in text_links:
        url = txt_link.get('href')
        if 'txt' in url:
            print(link+url)
            text_result = requests.get(link+url)
            text_page = text_result.content
            text_soup = BeautifulSoup(text_page)
            dates = url.split('.')[1]
            text = text_soup.get_text()
            texts.append({'dates': dates, 'text': text})

In [4]:
# # Turn our web scraped data in a dataframe
humanist_vols = pd.DataFrame.from_dict(texts, orient='columns')

In [10]:
# Write to csv
humanist_vols.to_csv('web_scraped_humanist_listserv.csv')